**Amazon Simple Storage Service (Amazon S3)** is an object storage service that offers scalability, data availability, security, and performance.


Amazon S3 is designed for 99.999999999% (11 9's) of durability, and stores data for millions of applications for companies all around the world.


An **Amazon S3 bucket** is a storage location to hold files. S3 files are referred to as **objects**.



**Boto 3** Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.


**Create an Amazon S3 bucket**

The name of an Amazon S3 bucket must be unique across all regions of the AWS platform. The bucket can be located in a specific region to minimize latency or to address regulatory requirements.

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError

In [ ]:
AWS_ACCESS_KEY_ID = ''
AWS_SECRET_ACCESS_KEY = ''

s3_client = boto3.client('s3', 'us-east-2',
                          aws_access_key_id = AWS_ACCESS_KEY_ID,
                          aws_secret_key_id = AWS_SECRET_ACCESS_KEY)

location = {'LocationConstraint': 'us-east-2'}
s3_client.create_bucket(Bucket = 'mindy-coursera', 
CreateBucketConfiguration = location)

## Listing Buckets

In [ ]:
response = s3_client.list_buckets()

for bucket in response['Buckets']:
    print(bucket['Name'])

## Uploading files


In [ ]:
file_name = 'flower_image.jpeg'
bucket_name = 'mindy-coursera'
object_name = file_name

response = s3_client.upload_file(file_name, bucket_name, object_name)

## Upload as File Object

In [ ]:
with open(file_name, 'rb') as f:
    s3_client.upload_fileobj(f, bucket_name, 'flower_image_file_obj.jpeg')

## Extra Args
Both upload_file and upload_fileobj accept an optional ExtraArgs parameter that can be used for various purposes.

Some Important ExtraArgs


In [ ]:
file_name = 'flower_image.jpeg'
bucket_name = 'mindy-coursera'
object_name = 'flower_image_public.jpeg'

response = s3_client.upload_file(file_name, bucket_name, object_name, 
                                 ExtraArgs = {'ACL':'public-read'})

## Downloading files

The methods provided by the AWS SDK for Python to download files are similar to those provided to upload files.


The download_file method accepts the names of the bucket and object to download and the filename to save the file to.

In [ ]:
s3_client.download_file(bucket_name, 'flower_image_public.jpeg', 'flower_image_s3.jpeg')

In [ ]:
with open('flower_image_write.jpeg', 'wb') as f:
    s3_client.download_fileobj(bucket_name, object_name, f)

## File transfer configuration


When uploading, downloading, or copying a file or S3 object, the AWS SDK for Python automatically manages retries and multipart and non-multipart transfers.

The management operations are performed by using reasonable default settings that are well-suited for most scenarios. To handle a special case, the default settings can be configured to meet requirements.

## Multipart transfers

Multipart transfers occur when the file size exceeds the value of the multipart_threshold attribute.


In [ ]:
from boto3.s3.transfer import TransferConfig 
GB = 1024**3
config = TransferConfig(multipart_threshold = 5*GB)
s3_client.upload_file(file_name, bucket_name, 'flower_multi_transfer.jpeg', Config = config)

## Presigned URLs

A user who does not have AWS credentials or permission to access an S3 object can be granted temporary access by using a presigned URL.

A presigned URL is generated by an AWS user who has access to the object. The generated URL is then given to the unauthorized user. The presigned URL can be entered in a browser or used by a program or HTML webpage. The credentials used by the presigned URL are those of the AWS user who generated the URL.

A presigned URL remains valid for a limited period of time which is specified when the URL is generated.

In [ ]:
response_pre_signed = s3_client.generate_presigned_url('get_object', 
Params = {'Bucket':bucket_name, 'Key':'flower_multi_transfer.jpeg'}, 
ExpiresIn = 3600)
print(response_pre_signed)

## Bucket policies

An S3 bucket can have an optional policy that grants access permissions to other AWS accounts or AWS Identity and Access Management (IAM) users. Bucket policies are defined using the same JSON format as a resource-based IAM policy.

## Retrieve a Bucket Policy

In [ ]:
result = s3_client.get_bucket_policy(Bucket = 'mindy-coursera')
print(result['Policy'])

## Set a bucket policy

A bucket's policy can be set by calling the put_bucket_policy method.

The policy is defined in the same JSON format as an IAM policy. 



## Policy Format

The **Sid (statement ID)** is an optional identifier that you provide for the policy statement. You can assign a Sid value to each statement in a statement array.

The **Effect** element is required and specifies whether the statement results in an allow or an explicit deny. Valid values for Effect are Allow and Deny.

By default, access to resources is denied. 

Use the **Principal** element in a policy to specify the principal that is allowed or denied access to a resource.

You can specify any of the following principals in a policy:

- AWS account and root user
- IAM users
- Federated users (using web identity or SAML federation)
- IAM roles
- Assumed-role sessions
- AWS services
- Anonymous users


The **Action** element describes the specific action or actions that will be allowed or denied. 

We specify a value using a service namespace as an action prefix (iam, ec2, sqs, sns, s3, etc.) followed by the name of the action to allow or deny.

The **Resource** element specifies the object or objects that the statement covers. We specify a resource using an ARN. Amazon Resource Names (ARNs) uniquely identify AWS resources.

Let's define a policy that enables any user to retrieve any object stored in the bucket identified by the bucket_name variable.

In [ ]:
import json
bucket_name = 'mindy-coursera'
bucket_policy = {
    'Version':'2012-10-17',
    'Statement':[{
        'Sid':'AddPerm',
        'Effect':'Allow',
        'Principal':'*',
        'Action':['s3:GetObject'],
        'Resource':f'arn:aws:s3:::{bucket_name}/*'
    }]
}

bucket_policy = json.dumps(bucket_policy)
s3_client.put_bucket_policy(Bucket = bucket_name, Policy = bucket_policy)

In [ ]:
result = s3_client.get_bucket_policy(Bucket = 'mindy-coursera')
print(result['Policy'])

## Delete a bucket policy


In [ ]:
s3_client.delete_bucket_policy(Bucket = 'mindy-coursera')

## CORS Configuration

Cross Origin Resource Sharing (CORS) enables client web applications in one domain to access resources in another domain. An S3 bucket can be configured to enable cross-origin requests. The configuration defines rules that specify the allowed origins, HTTP methods (GET, PUT, etc.), and other elements.

## Retrieve a bucket CORS configuration

Retrieve a bucket's CORS configuration by calling the AWS SDK for Python get_bucket_cors method.


In [ ]:
response = s3_client.get_bucket_cors(Bucket = 'mindy-coursera')
print(response(['CORSRules']))

## Set Bucket CORS

In [ ]:
cors_configuration = {
    'CORSRules':[
        {'AllowedHeaders':['Authorization'],
         'AllowedMethods':['GET', 'PUT'],
         'AllowedOrigins':["*"],
         'ExposeHeaders':['GET', 'PUT'],
         'MaxAgeSeconds':3000
        }
    ]
}

s3_client.put_bucket_cors(Bucket = bucket_name, CORSConfiguration = cors_configuration)

In [ ]:
response = s3_client.get_bucket_cors(Bucket = 'mindy-coursera')
print(response(['CORSRules']))